In [1]:
from preprocessing import Preproc
import pandas as pd
import numpy as np

In [ ]:
import time

In [ ]:

series = ['airline']
for serie in series:
    path_serie = f'/home/eraylson/union/timeseries/{serie}.txt'
    df_serie = pd.read_csv(path_serie, header=None)
    pre = Preproc()
    serie_np = df_serie.to_numpy().reshape(-1, )
    dict_samples = pre.split_serie(serie_np, perc_train = 0.75)
    train, test = dict_samples['train'], dict_samples['test']
    _ = pre.normalise_interval(train)
    train_norm = pre.normalise_interval(train)
    test_norm = pre.normalise_interval(test)
    idx_lags = pre.select_lag_acf( train_norm, 20)
    X_train, y_train = pre.create_windows(train_norm, np.max(idx_lags)+1)

    qtd_lags = np.max(idx_lags)
    test_with_previous = np.vstack([train_norm[-qtd_lags-1:], test_norm])
    X_test, y_test = pre.create_windows(test_with_previous, np.max(idx_lags)+1)
    
    X_train = X_train[:, idx_lags]
    X_test = X_test[:, idx_lags]

    from ensemble import Ensemble
    ens = Ensemble(100, 'KNN', approach_generate='bagging', serie_name = serie)
    start = time.time()
    ens.fit(X_train, y_train)
    end = time.time()
    duration = end - start
    #ens.save_models(serie, 'lstm')
    with open(f'{serie}_pool_lstm_time.txt', 'w') as f:
        f.write(f"tempo de execução: {duration}")

#6.1s

In [ ]:
ens.pool['models']

In [ ]:
'''[SVR(C=0.1, epsilon=0.0001, gamma=70.0),
 SVR(C=0.1, epsilon=0.0001, gamma=70.0),
 SVR(C=0.1, epsilon=0.0001, gamma=70.0),
 SVR(C=0.1, epsilon=0.0001, gamma=70.0),
 SVR(C=0.1, epsilon=0.0001, gamma=70.0)]'''

In [ ]:
eqweqwe

In [ ]:
train_norm = pre.normalise_interval(train)
test_norm = pre.normalise_interval(test)

In [ ]:
idx_lags = pre.select_lag_acf( train_norm, 20)

In [ ]:
X_train, y_train = pre.create_windows(train_norm, np.max(idx_lags)+1)


In [ ]:
qtd_lags = np.max(idx_lags)
test_with_previous = np.vstack([train_norm[-qtd_lags-1:], test_norm])
X_test, y_test = pre.create_windows(test_with_previous, np.max(idx_lags)+1)

In [ ]:
X_train = X_train[:, idx_lags]
X_test = X_test[:, idx_lags]

In [ ]:
from ensemble import Ensemble

In [ ]:
ens = Ensemble(100, 'SVR', approach_generate='bagging')

In [ ]:
ens.fit(X_train, y_train)

In [ ]:
ens.pool

In [ ]:
ens.save_models('airline', 'svr')

In [ ]:
from joblib import load

In [ ]:
model = load('/home/eraylson/union/airline/svr/model_0.joblib')

In [ ]:
model.predict(X_test[0].reshape(1, -1))

In [ ]:
X_test[0].shape

In [ ]:
len(X_test[0].shape)

In [ ]:
X_test[0].reshape(1, -1).shape

In [ ]:
X_test.shape

In [ ]:
ens.predict(X_test[0])

In [ ]:
y_pred = ens.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt 

plt.plot(y_pred, label = 'pred')
plt.plot(y_test, label = 'target')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
MSE(y_pred, y_test) #0.02836